In [3]:
from langgraph.prebuilt import create_react_agent

def get_weather(city:str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_react_agent(
    model = "google_genai:gemini-2.0-flash",
    tools = [get_weather],
    prompt = "You are a helpful asssistant"
)

agent.invoke(
    {
        "messages" : [{
            "role" : "user",
            "content" : "what is the weather in sf"
    }]
    }
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='958769bf-6fcd-446d-a44f-f6c20ce17406'),
  AIMessage(content='Could you please spell out SF?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c7ab5a70-3b15-4c39-ac23-52ac78d1e385-0', usage_metadata={'input_tokens': 26, 'output_tokens': 8, 'total_tokens': 34, 'input_token_details': {'cache_read': 0}})]}

In [4]:
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent

model = init_chat_model(
    model = "google_genai:gemini-2.0-flash",
    temperature = 0
)

agent = create_react_agent(
    model = model,
    tools = [get_weather]
)

In [8]:
from langgraph.prebuilt import create_react_agent

gemini_model = 'google_genai:gemini-2.0-flash'

agent = create_react_agent(
    model = gemini_model,
    tools = [get_weather],
    prompt = 'Never answer questions about the weather'
)

agent.invoke(
    {'messages':[{
        "role" : "user",
        "content": "what is the weather in Sanfrancisco" }]}
)

{'messages': [HumanMessage(content='what is the weather in Sanfrancisco', additional_kwargs={}, response_metadata={}, id='ea393962-3dc3-48d4-9f23-7d3c9d171eff'),
  AIMessage(content='I am sorry, I cannot fulfill this request. The available tools lack the desired functionality.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--51d73e2d-97c6-40ac-98cb-808555bfe5cc-0', usage_metadata={'input_tokens': 28, 'output_tokens': 19, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})]}

In [21]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

def prompt(state : AgentState, config : RunnableConfig) -> list[AnyMessage]:
    user_name = config['configurable'].get('user_name')
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role" : "system", "content" : system_msg}] + state['messages']

agent = create_react_agent(
    model = gemini_model,
    tools = [get_weather],
    prompt = prompt
)

agent.invoke(
    {'messages':[
        {
            "role" : "user",
            "content" : "what is the weather in Sanfrancisco"}
    ]},
    config={"configurable" : {"user_name" : "John Smith"}}
)

{'messages': [HumanMessage(content='what is the weather in Sanfrancisco', additional_kwargs={}, response_metadata={}, id='98dbaeaa-a6df-44aa-add6-dc0a5e27e49d'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "Sanfrancisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--903a4f29-2750-4142-8a8a-4d06b7ec2836-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Sanfrancisco'}, 'id': '02227f6e-c710-4cb1-8713-6ca8186716dd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 35, 'output_tokens': 7, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in Sanfrancisco!", name='get_weather', id='0332d0a3-edd9-45e0-a85c-94f55d6fe0bb', tool_call_id='02227f6e-c710-4cb1-8713-6ca8186716dd'),
  AIMessage(content='Hello John Smith, the weather in Sanfranc

In [28]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model = gemini_model,
    tools = [get_weather],
    checkpointer = checkpointer
)

config = {'configurable' : {"thread_id":"1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sanfrancisco"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    {'configurable' : {"thread_id":"2"}}
)

In [29]:
sf_response

{'messages': [HumanMessage(content='what is the weather in sanfrancisco', additional_kwargs={}, response_metadata={}, id='53b272cc-e6ec-485e-a7c0-ba831501b093'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "sanfrancisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--d35784ee-4514-4c9e-a64c-f2bb5a1cc87f-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sanfrancisco'}, 'id': 'd20c1603-f282-468b-b5a0-3ff8d7dd6424', 'type': 'tool_call'}], usage_metadata={'input_tokens': 22, 'output_tokens': 7, 'total_tokens': 29, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in sanfrancisco!", name='get_weather', id='0c98d986-95f6-4748-9920-c4cc5914d040', tool_call_id='d20c1603-f282-468b-b5a0-3ff8d7dd6424'),
  AIMessage(content="It's always sunny in sanfrancisco!", addi

In [27]:
ny_response

{'messages': [HumanMessage(content='what about new york?', additional_kwargs={}, response_metadata={}, id='719c3d28-fb7f-4bb4-951d-4ae52fb710fb'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "new york"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--6f0b3558-baeb-49fb-ad08-7967f98be4d1-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'new york'}, 'id': 'f0d3db8c-3005-462f-880b-092cb7b04e1b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 19, 'output_tokens': 6, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in new york!", name='get_weather', id='95a8b0d2-d979-4271-9f80-520083067501', tool_call_id='f0d3db8c-3005-462f-880b-092cb7b04e1b'),
  AIMessage(content="It's always sunny in new york!", additional_kwargs={}, response_meta

In [32]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions : str

agent = create_react_agent(
    model = gemini_model,
    tools = [get_weather],
    response_format = WeatherResponse
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in Boston"}]}
)

response["structured_response"]

WeatherResponse(conditions='sunny')

In [33]:
response

{'messages': [HumanMessage(content='what is the weather in Boston', additional_kwargs={}, response_metadata={}, id='10a884b5-e431-4f69-9e88-8984f7d9fd5b'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"city": "Boston"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--2687db83-b033-4829-982a-c23d5d867aec-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Boston'}, 'id': 'bbce3159-c200-458a-ba6c-f7fc3e94b992', 'type': 'tool_call'}], usage_metadata={'input_tokens': 20, 'output_tokens': 5, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="It's always sunny in Boston!", name='get_weather', id='acd14d8b-2d39-4e0d-84ed-817c9080e8eb', tool_call_id='bbce3159-c200-458a-ba6c-f7fc3e94b992'),
  AIMessage(content="It's always sunny in Boston!", additional_kwargs={}, response_met

In [36]:
response["structured_response"].conditions

'sunny'